In [5]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os 
from os import path
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

def colorselection(image,low,high):
    hsv=cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, low, high) 
    return cv2.bitwise_and(image,image, mask= mask)

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    x1left=x2left=x1right=x2right=0
    y1left=y1right=y2left=y2right=0
    nblineleft=nblineright=0
    for line in lines:
        for x1,y1,x2,y2 in line:
            angle = (np.rad2deg(np.arctan2(y1 - y2, x1 - x2)))
         
            if (angle<-120.0 and angle>-150):
                #cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                x1left+=x1
                x2left+=x2
                y1left+=y1
                y2left+=y2
                nblineleft+=1
            elif (angle>=120.0 and angle<=150.0):
                   #right.append((x1,y1,x2,y2,xbottom))
                x1right+=x1
                x2right+=x2
                y1right+=y1
                y2right+=y2
                nblineright+=1
                #cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    x1right=x1right/nblineright
    x2right=x2right/nblineright
    y1right=y1right/nblineright
    y2right=y2right/nblineright
    x1left=x1left/nblineleft
    x2left=x2left/nblineleft
    y1left=y1left/nblineleft
    y2left=y2left/nblineleft
    
    if nblineleft!=0:
        cv2.line(img, (x1left, y1left), (x2left, y2left), color, thickness)
    if nblineright!=:
        cv2.line(img, (x1right, y1right), (x2right, y2right), color, thickness)
    return               

            
def line_properties(x1,y1,x2,y2):
    slope = (x2-x1)/(y2-y1)
    intercept =y1-(slope*x1)
    #xintersect = -intercept/slope
    return slope,intercept
            

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    color=(255, 0, 0)
    draw_lines(line_img, lines,color,5)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(path):
    #reading in an image
    #print('Path : ',path)
    if type(path) == str :
        image = mpimg.imread(path)
    else :
        image = path
    #printing out some stats and plotting
    print('This image is:', type(image), 'with dimesions:', image.shape)
    #plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image
    
    #Color selection
    #white = cv2.inRange(image, (0, 0, 180), (255, 25, 255)) 
    #yellow = cv2.inRange(image, (20, 80, 80), (50, 255, 255))
    #Converting image to grayscale
    grayimg = grayscale(image)
    #plt.imshow(grayimg, cmap='gray')

    #Smoothing the image with gaussian noise using correlation
    kernel_size=3
    blurgrayimg = gaussian_blur(grayimg, kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 100
    
    #http://stackoverflow.com/questions/21324950/how-to-select-the-best-set-of-parameters-in-canny-edge-detection-algorithm-imple
    high_tresh,im_tresh = cv2.threshold(blurgrayimg, 0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    high_threshold = high_tresh
    low_threshold = high_tresh * 0.5
    #print ('low_threshold : ',low_threshold)
    #print ('high_treshold : ', high_threshold)
    
    cnedges=canny(blurgrayimg,low_threshold,high_threshold)

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    #vertices = np.array([[(0,imshape[0]),(0, 0), (imshape[1], 0), (imshape[1],imshape[0])]], dtype=np.int32)
    vertices = np.array([[[460,320],[500,320],[900,550],[100,550]]],dtype=np.int32)
    roi = region_of_interest(cnedges,vertices)
    #plt.imshow(roi, cmap='gray')

    # Definmpimg.imshow(masked_edges)e the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 50 #minimum number of pixels making up a line
    max_line_gap =  50   # maximum gap in pixels between connectable line segments

    hgimg = hough_lines(roi, rho, theta, threshold, min_line_length, max_line_gap)
    #plt.imshow(hgimg, cmap='gray')


    #coledges = np.dstack((cnedges,cnedges,cnedges))
    lineimage = weighted_img(hgimg, image, α=0.8, β=1, λ=0.)
    #plt.figure()
    #plt.imshow(lineimage)
    return lineimage
    
def processBatchImage(dirPath):
    files = [x for x in os.listdir(dirPath) if path.isfile(dirPath+os.sep+x)]
    for f in files :
        plt.figure()
        plt.imshow(process_image(dirPath+os.sep+f))
    
    return 


def process_video(output,input) :
    white_output = output
    clip1 = VideoFileClip(input)
    white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(white_output, audio=False)

    return """
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(white_output)

#'''Process the test images'''
#processBatchImage("test_images/")

#'''Process The White Right video'''
#HTML(process_video('white.mp4',"solidWhiteRight.mp4"))

#'''Process The Yellow Left video'''
#HTML(process_video('yellow.mp4',"solidYellowLeft.mp4"))

#'''Process The Challenge video'''
#HTML(process_video('extra.mp4',"challenge.mp4"))


SyntaxError: invalid syntax (<ipython-input-5-05d070da12bd>, line 111)

In [ ]:
def colorselection(image,low,high):
    hsv=cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, low, high) 
    return cv2.bitwise_and(image,image, mask= mask)

def process_img(path):
    if type(path) == str :
        image = mpimg.imread(path)
    else :
        image = path
    #image = mpimg.imread('test_images/solidYellowLeft.jpg')
    #Color selection
    #white = cv2.inRange(image, (0, 0, 180), (255, 25, 255)) 
    #yellow = cv2.inRange(image, (20, 80, 80), (50, 255, 255))

    low_threshold=np.array([0,0,200])
    high_threshold=np.array([255,25,255])
    white = colorselection(image,low_threshold,high_threshold)
    #plt.figure()
    #plt.imshow(white)


    #image = mpimg.imread('test_images/solidYellowLeft.jpg')
    low_threshold=np.array([20,100,100])
    high_threshold=np.array([50,255,255])
    yellow = colorselection(image,low_threshold,high_threshold)
    #plt.figure()
    #plt.imshow(yellow)

    add = cv2.add(yellow,white)
    #plt.figure()
    #plt.imshow(add)
    #grayimg = grayscale(image)
    #plt.imshow(grayimg, cmap='gray')

    #Smoothing the image with gaussian noise using correlation
    #kernel_size=3
    #blurgrayimg = gaussian_blur(grayimg, kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 100
    high_threshold = 250

    cnedges=canny(add,low_threshold,high_threshold)
    #plt.figure()
    #plt.imshow(cnedges,cmap='gray')
    
    #vertices = np.array([[[460,320],[500,320],[900,550],[100,550]]],dtype=np.int32)
    vertices = np.array([[[(image.shape[1]/2)+50,(image.shape[0]+20)/2],[(image.shape[1]/2)+100,\
                           (image.shape[0]+20)/2],\
                          [image.shape[1],image.shape[0]],[0,image.shape[0]]]],dtype=np.int32)
    roi = region_of_interest(cnedges,vertices)
    #plt.figure()
    #plt.imshow(roi, cmap='gray')

    # Definmpimg.imshow(masked_edges)e the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 80    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap =  50   # maximum gap in pixels between connectable line segments

    hgimg = hough_lines(roi, rho, theta, threshold, min_line_length, max_line_gap)
    #plt.figure()
    #plt.imshow(hgimg, cmap='gray')


    #coledges = np.dstack((cnedges,cnedges,cnedges))
    lineimage = weighted_img(hgimg, image, α=0.8, β=1, λ=0.)
    #plt.figure()
    #plt.imshow(lineimage)
    return lineimage

def processBatchImg(dirPath):
    files = [x for x in os.listdir(dirPath) if path.isfile(dirPath+os.sep+x)]
    for f in files :
        plt.figure()
        plt.imshow(process_image(dirPath+os.sep+f))
    
    return 

def process_vid(output,input) :
    white_output = output
    clip1 = VideoFileClip(input)
    white_clip = clip1.fl_image(process_img) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(white_output, audio=False)

    return """
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(white_output)



In [6]:
image =process_img('challenge_frame.jpg')
plt.figure()
plt.imshow(image)

NameError: name 'process_img' is not defined

In [7]:
#processBatchImg("test_images/")

In [ ]:
HTML(process_vid('extra.mp4',"challenge.mp4"))

In [ ]:
#'''Process the test images'''
processBatchImage("test_images/")

In [ ]:

#'''Process The White Right video'''
HTML(process_vid('white.mp4',"solidWhiteRight.mp4"))

In [ ]:
#'''Process The Yellow Left video'''
HTML(process_vid('yellow.mp4',"solidYellowLeft.mp4"))


# Reflections

The pipeling built so far should be enhanced at the Draw_Lines() function's level . To address the challenge in last topic of this project. In fact when i run the challenge video against my program i notice many lines in addition to lane lines are annotated. To resolve this issue, i propose the following intuition:
*/for each line, calculate the slope angle and compare it to the lane lines (i made an assumption for them) 
*/compare the maximum X2 of each to line lane

Work out :
I added some instruction to the Draw_lines() function to address the first step of the intuition. It still remains the second step to do.

In [ ]:
#'''Process The Challenge video'''
#HTML(process_video('extra.mp4',"challenge.mp4"))